
# Cross-sectional pipeline walkthrough

This notebook runs the full workflow: load price/volume data, compute features, estimate factor exposures, assemble the modeling dataset, generate out-of-sample predictions, and form decile portfolios. Update the paths in the first cell to point to your data.


In [1]:
from pathlib import Path
import logging
import pandas as pd

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

DIR_DATA = Path("../data")
DIR_STOCK_DATA = DIR_DATA / "stock_data"
PATH_META = DIR_DATA / "metadata_listed.csv"
PATH_META_DELISTED = DIR_DATA / "metadata_delisted.csv"

# Controls
P_SAMPLE = 0.10              # 10% of tickers
MAX_ROWS_PER_TICKER = 1500   # reduce to 500 if still heavy
PRICE_COL = "adj_close"
VOLUME_COL = "volume"

# Read metadata (small)
df_meta = pd.read_csv(PATH_META)
df_meta_delisted = pd.read_csv(PATH_META_DELISTED)

# Combine ticker universe
tickers = pd.concat(
    [df_meta[["Symbol"]], df_meta_delisted[["Symbol"]]],
    ignore_index=True
)["Symbol"].dropna().astype(str).unique()

# Deterministic ~10% sampling so you get the same subset each run
def keep_ticker(t: str, p: float) -> bool:
    return (hash(t) % 10_000) / 10_000 < p

tickers_sample = [t for t in tickers if keep_ticker(t, P_SAMPLE)]
logging.info(f"Selected {len(tickers_sample)} tickers out of {len(tickers)} (~{P_SAMPLE:.0%}).")

all_stocks = []
skipped = 0

# Read only needed columns for speed/memory
usecols = ["date", PRICE_COL, VOLUME_COL]

for symbol in tickers_sample:
    path_stock = DIR_STOCK_DATA / f"{symbol}.csv"
    if not path_stock.exists():
        skipped += 1
        continue

    try:
        df_stock = pd.read_csv(
            path_stock,
            usecols=lambda c: c in usecols,   # only load what we need
            parse_dates=["date"],
        )

        # Keep most recent rows (assumes file is date-sorted; if not, sorting after tail is cheap)
        if MAX_ROWS_PER_TICKER is not None and len(df_stock) > MAX_ROWS_PER_TICKER:
            df_stock = df_stock.tail(MAX_ROWS_PER_TICKER)

        df_stock = df_stock.sort_values("date")

        # Downcast for memory
        df_stock[PRICE_COL] = pd.to_numeric(df_stock[PRICE_COL], errors="coerce", downcast="float")
        df_stock[VOLUME_COL] = pd.to_numeric(df_stock[VOLUME_COL], errors="coerce", downcast="float")

        # Set MultiIndex like your pipeline expects
        df_stock["ticker"] = symbol
        df_stock = df_stock.set_index(["ticker", "date"]).sort_index()

        all_stocks.append(df_stock)

    except Exception as e:
        skipped += 1
        logging.warning(f"Skipping {symbol}: {e}")

df_stocks = pd.concat(all_stocks) if all_stocks else pd.DataFrame()
logging.info(f"Loaded panel shape: {df_stocks.shape}, skipped: {skipped}, rows: {len(df_stocks):,}")

df_stocks.head()

2025-12-23 12:54:42,031 [INFO] Selected 1362 tickers out of 13888 (~10%).
2025-12-23 12:55:24,492 [INFO] Loaded panel shape: (1450777, 2), skipped: 238, rows: 1,450,777


adj_close     volume
ticker date                            
AA     2019-11-12  22.180000  3105100.0
       2019-11-13  21.219999  4458400.0
       2019-11-14  20.990000  2919400.0
       2019-11-15  20.889999  3289100.0
       2019-11-18  20.760000  3616500.0

In [2]:
from pathlib import Path
import logging

from features import FeatureConfig, compute_features
from factor_exposures import ExposureConfig, compute_factor_exposures
from final_dataset import assemble_modeling_dataset
from oos_predictions import generate_oos_predictions_all_models
from portfolios import compute_decile_portfolio_returns
from training_scheme import WindowConfig

# Paths
DIR_OUTPUT = Path("outputs")
DIR_OUTPUT.mkdir(parents=True, exist_ok=True)

# Configs
feature_cfg = FeatureConfig()
feature_cfg.price_col = getattr(feature_cfg, "price_col", "adj_close")
feature_cfg.volume_col = getattr(feature_cfg, "volume_col", "volume")

exposure_cfg = ExposureConfig()
window_cfg = WindowConfig()
standardize_method = "zscore"

# Rename to whatever the rest of your pipeline expects (your earlier code used stock_data)
stock_data = df_stocks

stock_data.head()

adj_close     volume
ticker date                            
AA     2019-11-12  22.180000  3105100.0
       2019-11-13  21.219999  4458400.0
       2019-11-14  20.990000  2919400.0
       2019-11-15  20.889999  3289100.0
       2019-11-18  20.760000  3616500.0

In [3]:
# Compute features
features = compute_features(stock_data, config=feature_cfg)
features.to_csv(DIR_OUTPUT / "features.csv")
features.head()

size  value  momentum  volatility  \
ticker date                                                                    
AA     2019-11-30 23:59:59.999999999  17.905862    NaN       NaN         0.0   
       2019-12-31 23:59:59.999999999  18.161554    NaN       NaN         0.0   
       2020-01-31 23:59:59.999999999  18.418282    NaN       NaN         0.0   
       2020-02-29 23:59:59.999999999  18.194516    NaN       NaN         0.0   
       2020-03-31 23:59:59.999999999  17.923060    NaN       NaN         0.0   

                                      investment  profitability  
ticker date                                                      
AA     2019-11-30 23:59:59.999999999         NaN            NaN  
       2019-12-31 23:59:59.999999999         NaN            NaN  
       2020-01-31 23:59:59.999999999         NaN            NaN  
       2020-02-29 23:59:59.999999999         NaN            NaN  
       2020-03-31 23:59:59.999999999         NaN            NaN

In [4]:
from pathlib import Path
import pandas as pd
import re

RAW_FACTOR_PATH = DIR_DATA / "F-F_Research_Data_Factors.csv"
CLEAN_FACTOR_PATH = DIR_DATA / "factors.csv"

# Read file as text lines
lines = RAW_FACTOR_PATH.read_text(errors="ignore").splitlines()

monthly_rows = []
in_monthly = False

for line in lines:
    s = line.strip()

    # Start when we see the monthly header
    if re.match(r"^,?\s*Mkt-RF\s*,\s*SMB\s*,\s*HML\s*,\s*RF\s*$", s):
        in_monthly = True
        continue

    # Stop when annual section begins
    if "Annual Factors" in s:
        break

    if not in_monthly:
        continue

    # Keep only YYYYMM rows
    if re.match(r"^\d{6}\s*,", s):
        monthly_rows.append(s)

# Build DataFrame from extracted CSV rows
csv_text = "date,Mkt-RF,SMB,HML,RF\n" + "\n".join(monthly_rows)
ff = pd.read_csv(pd.io.common.StringIO(csv_text))

# Convert to your expected format
ff = ff.rename(columns={"Mkt-RF": "MKT"})
ff["date"] = pd.to_datetime(ff["date"], format="%Y%m") + pd.offsets.MonthEnd(0)
ff = ff.set_index("date").sort_index()

for col in ["MKT", "SMB", "HML", "RF"]:
    ff[col] = pd.to_numeric(ff[col], errors="coerce") / 100.0

ff = ff[["MKT", "SMB", "HML", "RF"]]

# Save clean file
ff.to_csv(CLEAN_FACTOR_PATH)

ff.tail()

,MKT,SMB,HML,RF
date,,,,
2025-06-30,0.0486,0.0083,-0.0160,0.0034
2025-07-31,0.0198,0.0027,-0.0127,0.0034
2025-08-31,0.0184,0.0387,0.0442,0.0038
2025-09-30,0.0339,-0.0185,-0.0105,0.0033
2025-10-31,0.0195,-0.0055,-0.0310,0.0037


In [5]:
# Load factors
factors = pd.read_csv(CLEAN_FACTOR_PATH, parse_dates=["date"]).set_index("date").sort_index()
factors.head()

,MKT,SMB,HML,RF
date,,,,
1926-07-31,0.0289,-0.0255,-0.0239,0.0022
1926-08-31,0.0264,-0.0114,0.0381,0.0025
1926-09-30,0.0038,-0.0136,0.0005,0.0023
1926-10-31,-0.0327,-0.0014,0.0082,0.0032
1926-11-30,0.0254,-0.0011,-0.0061,0.0031


In [6]:
# Compute monthly returns and factor exposures
monthly_price = (
    stock_data[feature_cfg.price_col]
    .groupby(level="ticker")
    .resample("ME", level="date")
    .last()
)
returns = (
    monthly_price
    .groupby(level="ticker")
    .pct_change(fill_method=None)
    .rename("RET")
    .to_frame()
)
returns.index = returns.index.set_names(["ticker", "date"])
returns = returns.dropna(subset=["RET"])

exposures = compute_factor_exposures(returns, factors, config=exposure_cfg)
exposures.to_csv(DIR_OUTPUT / "factor_exposures.csv")
exposures.groupby(level="ticker").apply(lambda x: x.dropna().index.get_level_values("date").min()).head()
returns.head(), exposures.head()

(                        RET
 ticker date                
 AA     2019-12-31  0.057002
        2020-01-31 -0.351464
        2020-02-29 -0.005735
        2020-03-31 -0.555876
        2020-04-30  0.263750,
                    beta_MKT  beta_SMB  beta_HML
 ticker date                                    
 AA     2022-11-30       NaN       NaN       NaN
        2022-12-31  2.409768  0.850754  1.471426
        2023-01-31  2.397537  0.851496  1.474260
        2023-02-28  2.393224  0.827739  1.481545
        2023-03-31  2.399850  0.818432  1.483439)

In [15]:
import pandas as pd

features = features.copy()

tickers = features.index.get_level_values("ticker")
dates = pd.DatetimeIndex(features.index.get_level_values("date"))

# normalize to month-end midnight (00:00:00)
dates_norm = dates.to_period("M").to_timestamp("M")

features.index = pd.MultiIndex.from_arrays(
    [tickers, dates_norm],
    names=["ticker", "date"]
).sort_values()

features = features.sort_index()

# sanity check
features.index.get_level_values("date")[:5]

DatetimeIndex(['2019-11-30', '2019-12-31', '2020-01-31', '2020-02-29',
               '2020-03-31'],
              dtype='datetime64[ns]', name='date', freq=None)

In [20]:
# Assemble modeling dataset
panel = assemble_modeling_dataset(
    returns,
    features,
    exposures,
    standardize=standardize_method,
)
panel.to_csv(DIR_OUTPUT / "modeling_dataset.csv")
panel = panel.dropna(subset=["next_return"])
panel.head()

size     value  momentum  volatility  investment  \
ticker date                                                               
AA     2019-12-31  1.201701  0.000000       0.0         0.0         0.0   
       2020-01-31  1.285551  0.000000       0.0         0.0         0.0   
       2020-02-29  1.186957  0.000000       0.0         0.0         0.0   
       2020-03-31  0.992997  0.000000       0.0         0.0         0.0   
       2020-04-30  1.099926  0.413218       0.0         0.0         0.0   

                   profitability  beta_MKT  beta_SMB  beta_HML  next_return  
ticker date                                                                  
AA     2019-12-31            0.0       NaN       NaN       NaN    -0.351464  
       2020-01-31            0.0       NaN       NaN       NaN    -0.005735  
       2020-02-29            0.0       NaN       NaN       NaN    -0.555876  
       2020-03-31            0.0       NaN       NaN       NaN     0.263750  
       2020-04-30            0.0       NaN       NaN       NaN     0.130061

In [21]:
# Generate out-of-sample predictions
feature_cols = [c for c in panel.columns if c != "next_return"]
predictions = generate_oos_predictions_all_models(
    panel,
    feature_cols,
    target_col="next_return",
    window_config=window_cfg,
)
predictions.to_csv(DIR_OUTPUT / "predictions.csv")
predictions.head()

/opt/conda/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.683e-05, tolerance: 9.075e-06
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.683e-05, tolerance: 9.075e-06
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.683e-05, tolerance: 9.075e

ols_pred  ridge_pred  lasso_pred  elasticnet_pred  \
ticker date                                                            
AA     2022-12-31  1.203869    1.203200    0.340914         0.340914   
       2023-01-31  0.814506    0.967704    0.333494         0.333494   
       2023-02-28  0.865930    0.792710    0.320083         0.320083   
       2023-03-31 -1.476634   -1.454837    0.306994         0.306994   
       2023-04-30 -2.806751   -2.752302    0.296537         0.296537   

                   random_forest_pred  realized_return  
ticker date                                             
AA     2022-12-31           -0.050458         0.148889  
       2023-01-31            0.116444        -0.063170  
       2023-02-28            0.033530        -0.128794  
       2023-03-31           -0.031922        -0.127350  
       2023-04-30            0.051894        -0.143513

In [23]:
# Form decile portfolios
portfolios = compute_decile_portfolio_returns(
    predictions,
    return_col="realized_return",
)
portfolios.to_csv(DIR_OUTPUT / "decile_returns.csv")
portfolios.head()

elasticnet_pred                                            ...  \
                        1   2   3   4   5   6   7   8   9         10  ...   
date                                                                  ...   
1978-07-31             NaN NaN NaN NaN NaN NaN NaN NaN NaN  0.284144  ...   
1978-08-31             NaN NaN NaN NaN NaN NaN NaN NaN NaN  0.075000  ...   
1978-09-30             NaN NaN NaN NaN NaN NaN NaN NaN NaN -0.085273  ...   
1978-10-31             NaN NaN NaN NaN NaN NaN NaN NaN NaN  0.108579  ...   
1978-11-30             NaN NaN NaN NaN NaN NaN NaN NaN NaN  0.015384  ...   

           ridge_pred                                            
                   1   2   3   4   5   6   7   8   9         10  
date                                                             
1978-07-31        NaN NaN NaN NaN NaN NaN NaN NaN NaN  0.284144  
1978-08-31        NaN NaN NaN NaN NaN NaN NaN NaN NaN  0.075000  
1978-09-30        NaN NaN NaN NaN NaN NaN NaN NaN NaN -0.085273  
1978-10-31        NaN NaN NaN NaN NaN NaN NaN NaN NaN  0.108579  
1978-11-30        NaN NaN NaN NaN NaN NaN NaN NaN NaN  0.015384  

[5 rows x 50 columns]